# Importaciones

In [5]:
import datetime
import logging
import os
import pandas as pd

# Configurar logging

In [6]:
class Logging:

    @staticmethod
    def setup_logging():
        """Configura el sistema de logging para que use un único archivo y agregue nuevos registros."""
        log_filename = "data_analysis.log"

        logging.basicConfig(
            level=logging.INFO,
            format="%(asctime)s [%(levelname)s] %(message)s",
            handlers=[
                logging.FileHandler(log_filename, mode="a"),
                logging.StreamHandler()
            ]
        )
        return logging.getLogger(__name__)
    
    def report_data_issues(
        self,
        table_name: str,
        data_issues: dict
    ):
        """
        Crea un archivo CSV para registrar problemas de calidad de datos.

        :param table_name: Nombre de la tabla afectada.
        :param data_issues: DataFrame con los datos problemáticos.
        :param issue_type: Tipo de problema de datos (e.g., "duplicados", "valores no válidos").
        """
        # Crear el nombre del archivo con fecha y hora completa
        timestamp = datetime..datetime.now().strftime('%Y%m%d_%H%M%S')
        file_name = f"{table_name}_{timestamp}.csv"
        file_path = os.path.join(self.output_dir, file_name)

        try:
            # Guardar el DataFrame como un archivo CSV
            data_issues.to_csv(file_path, index=False)
            self.logger.info(f"Archivo CSV generado: {file_path}")
        except Exception as e:
            self.logger.error(f"Error al guardar el archivo CSV para {table_name}: {str(e)}")

## Pre-Analisis

In [7]:
REQUIRED_SHEETS = ["film", "inventory", "rental", "customer", "store"]


class DataProcessor:
    def __init__(self, logger: logging.Logger):
        self.logger = logger
        self.df = None

    def load_and_validate_excel(self, file_path: str, table: str):
        """
        Carga y valida un archivo Excel.

        :param file_path: Ruta del archivo Excel a cargar.
        """
        # Verificar si el archivo existe
        if not os.path.isfile(file_path):
            self.logger.error(f"El archivo '{file_path}' no existe o no es válido.")
            return None

        try:
            # Leer las hojas del archivo Excel
            all_sheets = pd.ExcelFile(file_path).sheet_names
            cleaned_sheets = [sheet.strip() for sheet in all_sheets]

            # Validar que las hojas requeridas estén presentes
            missing_sheets = [
                sheet for sheet in REQUIRED_SHEETS if sheet not in cleaned_sheets
            ]
            if missing_sheets:
                self.logger.error(
                    f"El archivo '{file_path}' no contiene las hojas requeridas: {missing_sheets}."
                )
                return None

            # Cargar cada hoja requerida en un DataFrame
            self.df = pd.read_excel(file_path, sheet_name=table)
            self.logger.info(f"Archivo '{file_path}' cargado correctamente.")

        except Exception as e:
            self.logger.error(f"Error al cargar el archivo Excel '{file_path}': {str(e)}")

    def dataframe_pre_analyze(self, table_name: str):
        """
        Analiza un DataFrame para detectar valores nulos, duplicados y tipos de
        datos y guarda el resultado en un diccionario y en el registro de logger.

        Args:
            df: pandas DataFrame a analizar
            table_name: nombre de la tabla para el reporte
        """
        self.logger.info(f"Iniciando pre-análisis de la tabla {table_name}")
        
        # Diccionario para almacenar el análisis
        analysis_report = {
            "table_name": table_name,
            "general_info": {
                "num_rows": len(self.df),
                "num_columns": len(self.df.columns)
            },
            "data_types": self.df.dtypes.to_dict(),
            "null_values": self.df.isnull().sum().to_dict(),
            "duplicates": {"num_duplicates": self.df.duplicated().sum()},
            "basic_stats": self.df.describe().to_dict()
        }

        # Valores únicos para columnas categóricas
        categorical_columns = self.df.select_dtypes(include=["object"]).columns
        unique_values = {col: self.df[col].nunique() for col in categorical_columns}
        analysis_report["unique_values"] = unique_values

        # Registro en logger
        self.logger.info(f"Reporte de pre-análisis: {analysis_report}")
        return analysis_report

    def clean_numeric_columns(self, table_name: str, numeric_columns: list):
        """
        Limpia las columnas numéricas, eliminando filas con valores no numéricos

        Args:
            table_name: nombre de la tabla
            numeric_columns: lista de columnas que deben ser numéricas
        """
        rows_before = len(self.df)
        non_numeric_values_dict = {}

        for column in numeric_columns:
            if column in self.df.columns:
                # Verificar valores no numéricos
                non_numeric_mask = pd.to_numeric(self.df[column], errors="coerce").isna()
                if non_numeric_mask.any():
                    non_numeric_values = self.df[non_numeric_mask][column].unique()
                    non_numeric_values_dict[f"{table_name}_{column}"] = non_numeric_values.tolist()
                    self.df.drop(self.df[non_numeric_mask].index, inplace=True)

                    # Convertir al tipo numérico adecuado
                    if any(keyword in column for keyword in ["id", "year", "length", "num_voted"]):
                        self.df[column] = pd.to_numeric(self.df[column], errors="coerce").astype("Int64")
                    else:
                        self.df[column] = pd.to_numeric(self.df[column], errors="coerce").astype("float64")

        if non_numeric_values_dict:
            self.logger.warning(f"Valores no numéricos encontrados en {table_name}: {non_numeric_values_dict}")
        
        rows_after = len(self.df)
        self.logger.info(f"Se eliminaron {rows_before - rows_after} filas con valores no numéricos en {table_name}")
        
        # Limpiar y convertir columnas de fecha
        date_columns = [col for col in self.df.columns if "date" in col.lower()]
        for col in date_columns:
            self.df[col] = pd.to_datetime(self.df[col].str.strip().str.lstrip("'\"").str.rstrip("'\""), errors='coerce')

    def analyze_dataframe(self, table_name: str):
        """
        Analiza un DataFrame para detectar valores nulos, duplicados y tipos de datos

        Args:
            table_name: nombre de la tabla para el reporte
        """
        self.logger.info(f"Dimensiones después de limpieza de {table_name}: {self.df.shape}")
        data_types = {column: dtype for column, dtype in self.df.dtypes.items()}
        self.logger.info(f"Tipos de datos en {table_name}: {data_types}")

        duplicates = self.df.duplicated().sum()
        if duplicates > 0:
            self.logger.warning(f"Se encontraron {duplicates} filas duplicadas")
        return self.df

    def verify_relationships(self, dfs: dict):
        """
        Verifica la integridad referencial entre las tablas

        Args:
            dfs: diccionario con los DataFrames {nombre_tabla: DataFrame}
        """
        self.logger.info("Verificando integridad referencial entre tablas")

        # Verificar integridad referencial entre film y inventory
        if "film" in dfs and "inventory" in dfs:
            invalid_films = set(dfs["inventory"]["film_id"]) - set(dfs["film"]["film_id"])
            if invalid_films:
                self.logger.error(f"Film_ids en inventory que no existen en film: {len(invalid_films)}")
                dfs["inventory"] = dfs["inventory"][~dfs["inventory"]["film_id"].isin(invalid_films)]

        # Verificar integridad referencial entre inventory y store
        if "inventory" in dfs and "store" in dfs:
            invalid_stores = set(dfs["inventory"]["store_id"]) - set(dfs["store"]["store_id"])
            if invalid_stores:
                self.logger.error(f"Store_ids en inventory que no existen en store: {len(invalid_stores)}")
                dfs["inventory"] = dfs["inventory"][~dfs["inventory"]["store_id"].isin(invalid_stores)]


### Analisis

In [8]:
logger = Logging().setup_logging()
processor = DataProcessor(logger)

try:
    numeric_columns = {
        "inventory": ["inventory_id", "film_id", "store_id"],
        "film": ["film_id", "release_year", "length", "num_voted_users", "language_id", "rental_rate", "replacement_cost", "rental_duration"],
        "rental": ["rental_id", "inventory_id", "customer_id", "staff_id"],
        "customer": ["customer_id"],
        "store": ["store_id", "address_id", "active"]
    }

    excel_file = "/home/dager/Documentos/Proyectos/Pruebas Técnica/Quind/Films_2 (3).xlsx"
    dataframes = {}

    for table in REQUIRED_SHEETS:
        try:
            logger.info(f"{'=' * 50}\nCargando tabla: {table}")
            processor.load_and_validate_excel(excel_file, table)

            # Pre-analizar la tabla
            processor.dataframe_pre_analyze(table)

            if table in numeric_columns:
                processor.clean_numeric_columns(table, numeric_columns[table])

            df = processor.analyze_dataframe(table)
            dataframes[table] = df

        except Exception as e:
            logger.error(f"Error al procesar la tabla {table}: {str(e)}")

    processor.verify_relationships(dataframes)

    output_file = f"cleaned_data_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx"
    with pd.ExcelWriter(output_file) as writer:
        for table_name, df in dataframes.items():
            df.to_excel(writer, sheet_name=table_name, index=False)
    logger.info(f"Datos limpios guardados en: {output_file}")

except Exception as e:
    logger.error(f"Error general en el procesamiento: {str(e)}")

2024-11-15 06:20:48,865 [INFO] ==================================================
Cargando tabla: film


2024-11-15 06:20:50,276 [INFO] Archivo '/home/dager/Documentos/Proyectos/Pruebas Técnica/Quind/Films_2 (3).xlsx' cargado correctamente.
2024-11-15 06:20:50,278 [INFO] Iniciando pre-análisis de la tabla film
2024-11-15 06:20:50,296 [INFO] Reporte de pre-análisis: {'table_name': 'film', 'general_info': {'num_rows': 1003, 'num_columns': 14}, 'data_types': {'film_id': dtype('O'), ' title': dtype('O'), ' description': dtype('O'), ' release_year': dtype('O'), ' language_id': dtype('int64'), ' original_language_id': dtype('O'), ' rental_duration': dtype('int64'), ' rental_rate': dtype('O'), ' length': dtype('O'), ' replacement_cost': dtype('O'), ' num_voted_users': dtype('O'), ' rating': dtype('O'), ' special_features': dtype('O'), 'last_update': dtype('O')}, 'null_values': {'film_id': 0, ' title': 0, ' description': 0, ' release_year': 0, ' language_id': 0, ' original_language_id': 0, ' rental_duration': 0, ' rental_rate': 0, ' length': 0, ' replacement_cost': 0, ' num_voted_users': 0, ' rat